In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import sympy

In [2]:
def scalar_mult(u, v):
  sum = 0
  for i in range(0, len(v)):
    sum += u[i]*v[i]
  return sum

In [3]:
def vmul(u, v):
  result = np.array([[0., 0.], [0., 0.]])
  for i in range(2):
    for j in range(2):
      result[i][j] = u[i][0] * v[j]
  return result


In [4]:
def v(H, W):
  v = np.array([0., 0.])
  for i in range(0, 2):
    for j in range(0, 2):
     v[i] +=  (H[i][j]) * (W[j])
  return v

In [5]:
def Powell_method(F, X, counter_F, counter_W, counter_A, counter_powell):
  g, o = 0.49, 0.3
  A = np.eye(2)
  W = w(X)
  counter_W += 1
  eps = 1e-3
  Xprev = np.array([0, 0])
  while (np.linalg.norm(X - Xprev) > eps):
    Xprev = X
    counter_powell += 1
    kappa = 1
    p = v(A, W)
    X_n = X + kappa * p
    while (f(X) - f(X_n)) < (o * kappa * scalar_mult(W, p)):
      kappa = kappa * g
      X_n = X + kappa * p
      counter_F += 1
    Wprev = W
    X = X_n

    W = w(X)
    counter_W += 1
    delta_X = X - Xprev
  #print('X', delta_X)
    delta_X_T = delta_X.reshape(-1, 1)
    delta_W = W - Wprev
  #print('W', delta_W)
    delta_W_T = delta_W.reshape(-1, 1)
    if (delta_X[0] == 0 and delta_X[1] == 0) or (delta_W[0] == 0 and delta_W[1] == 0) or counter_powell % 5 == 0:
      A = np.eye(2)
    else:
      delta_X = delta_X + v(A, delta_W)
      delta_X_T = delta_X.reshape(-1, 1)
      delta_A = -(vmul(delta_X_T, delta_X)) / (scalar_mult(delta_W, delta_X))
      A = A + delta_A
      counter_A += 1
  return X, counter_F, counter_W, counter_A, counter_powell

In [6]:




alpha = 200
X = np.array([-1, -2])
F = lambda x, y: alpha * (x ** 2 - y) ** 2 + (x - 1) ** 2 #Ф-я Розенброка
#def G(X):
  #return 3*X[0]**2/4 + 5*X[0]/4-2-X[1]
f = lambda X: F(X[0], X[1])
w = lambda X: np.array(-grad_F(X[0], X[1]))

x_, y_ = sympy.Symbol('x'), sympy.Symbol('y')
grad_F = sympy.tensor.array.derive_by_array(F(x_, y_), (x_, y_))
grad_F = sympy.lambdify((x_, y_), grad_F, 'numpy')

G = lambda x, y: 3*x**2/4 + 5*x/4-2-y
#G = lambda x, y: np.log(3*x**2/4 + 5*x/4-2-y)
#D = lambda x, y: 1 / G(x, y)
#D = lambda x, y: -G(x, y)
#d = lambda X: D(X[0], X[1])
g = lambda X: G(X[0], X[1])
grad_G = sympy.tensor.array.derive_by_array(G(x_, y_), (x_, y_))
grad_G = sympy.lambdify((x_, y_), grad_G, 'numpy')

eps = 1e-3
#счетчики
counter_powell = 0
iterations = 0 #итераций
counter_F = 0 #вычисленных функций
counter_W = 0 #вычисленных антиградиентов
counter_A = 0 #вычисленных матриц A


data = {f(X)}
counter_F += 1
X_line = [X[0]]
Y_line = [X[1]]

#gamma = 1.1

#r0 = abs(scalar_mult(grad_F(X[0], X[1]), grad_G(X[0], X[1])))/np.linalg.norm(grad_G(X[0], X[1]))**2;
r0 = 2
rk = r0
while True:
  iterations += 1
  Xprev = X
  #Fk = lambda X: f(X) + rk * (((g(X)+abs(g(X)))/2.0)**2)  #функция для минимизации
  Fk = lambda X: f(X) + rk * (max(0, g(X))**2)
  X, counter_F, counter_W, counter_A, counter_powell = Powell_method(Fk, X, counter_F, counter_W, counter_A, counter_powell) #градиентный спуск
  max_criteria = max(np.linalg.norm(X-Xprev), abs(f(X) - f(Xprev)))
  data.add(f(X))
  X_line.append(X[0])
  Y_line.append(X[1])
  if max_criteria < eps:
    break
  rk = rk + 2
print("=== Метод внешних штрафных функций ===")
print("eps = ", eps)
print("x* = ", X)
print("f(x*) = ", f(X))
print("Вызовов метода безусловной минимизации = ", iterations)
print("Вычислено функций = ", counter_F)
print("Вычислено градиентов = ", counter_W)
print("Вычислено матриц А = ", counter_A)
print("Итераций в методе Пауэлла = ", counter_powell)

=== Метод внешних штрафных функций ===
eps =  0.001
x* =  [0.94349147 0.89018879]
f(x*) =  0.0031932459254057615
Вызовов метода безусловной минимизации =  7
Вычислено функций =  394
Вычислено градиентов =  49
Вычислено матриц А =  33
Итераций в методе Пауэлла =  42
